<font color="green">**Function definitions for all semantic scores**</font>

**Import and download libraries**

In [ ]:
#Import libraries
import pandas as pd 
import numpy as np
import textstat
from textblob import TextBlob
import math
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [ ]:
#Download libraries
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adite\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Function to call all score computing functions**

In [ ]:
def compute_semantic_scores(df):
    df=compute_subjectivity_score(df)
    df=compute_polarity_score(df)
    df=compute_entropy_score(df)
    df=compute_flesch_index(df)
    df=compute_dale_chall_index(df)
    df=compute_sentiment_score(df)
    return df

**Function to compute subjectivity score**

In [ ]:
def compute_subjectivity_score(df):
    df['body']=df['body'].astype(str)
    df['subjectivity']=0
    length=len(df)
    for i in range(length):
        news=df.loc[i,"body"]
        subjectivityScore=TextBlob(news).sentiment.subjectivity
        df.loc[i,"subjectivity"]=subjectivityScore
    return df

**Function to compute polarity score**

In [ ]:
def compute_polarity_score(df):
    df['body']=df['body'].astype(str)
    df['polarity']=0
    length=len(df)
    for i in range(length):
        news=df.loc[i,"body"]
        polarityScore=TextBlob(news).sentiment.polarity
        df.loc[i,"polarity"]=polarityScore
    return df

**Function to compute entropy score**

In [ ]:
def compute_entropy_score(df):
    df['body']=df['body'].astype(str)
    df['entropy']=0
    length = len(df)
    for i in range(length):
        news = df.loc[i,"body"]
        fdist = FreqDist()
        for word in word_tokenize(news):
            fdist[word.lower()] += 1
        probs = [fdist.freq(l) for l in fdist]
        df.loc[i,"entropy"] = -sum(p * math.log(p,2) for p in probs)
    return df

**Function to compute flesch index**

In [ ]:
def compute_flesch_index(df):
    df['body']=df['body'].astype(str)
    df['flesch_index']=0
    length=len(df)
    for i in range(length):
        news=df.loc[i,"body"]
        fleschIndex=textstat.flesch_kincaid_grade(news)
        df.loc[i,"flesch_index"]=fleschIndex
    return df

**Function to compute dale chall index**

In [ ]:
def compute_dale_chall_index(df):
    df['body']=df['body'].astype(str)
    df['dale_chall_index']=0
    length=len(df)
    for i in range(length):
        news=df.loc[i,"body"]
        daleChallIndex=textstat.dale_chall_readability_score(news)
        df.loc[i,"dale_chall_index"]=daleChallIndex
    return df

**Function to compute sentiment score**

In [ ]:
def compute_sentiment_score(df):
    df['sentiment']=np.nan
    length=len(df)
    for i in range(length):
        polarity=df.loc[i,"polarity"]
        if polarity>=0.15:
            df.loc[i,"sentiment"]='positive'
        elif -0.15<polarity<0.15:
            df.loc[i,"sentiment"]='neutral'
        elif polarity<=-0.15:
            df.loc[i,"sentiment"]='negative'
    return df

**Function to call normalization functions for all scores**

In [ ]:
def normalize_semantic_scores(df):
    df.drop(['company','headlines','link','body'], axis=1,inplace=True)
    #df.drop(['company','headlines','body'], axis=1,inplace=True)
    df=normalize_polarity(df)
    df=normalize_entropy(df)
    df=normalize_flesch_index(df)
    df=normalize_dale_chall_index(df)
    return df

**Function to normalize polarity score**

In [ ]:
def normalize_polarity(df):
    df['normalized_polarity']=0
    length=len(df)
    minIndex=df['polarity'].min()
    maxIndex=df['polarity'].max()
    for i in range(length):
        index=df.loc[i,"polarity"]
        normalizedIndex=(index-minIndex)/(maxIndex-minIndex)
        df.loc[i,"normalized_polarity"]=normalizedIndex
    return df

**Function to normalize entropy score**

In [ ]:
def normalize_entropy(df):
    df['normalized_entropy']=0
    length=len(df)
    minIndex=df['entropy'].min()
    maxIndex=df['entropy'].max()
    for i in range(length):
        index=df.loc[i,"entropy"]
        normalizedIndex=(index-minIndex)/(maxIndex-minIndex)
        df.loc[i,"normalized_entropy"]=normalizedIndex
    return df

**Function to normalize flesch index**

In [ ]:
def normalize_flesch_index(df):
    df['normalized_flesch_index']=0
    length=len(df)
    minIndex=df['flesch_index'].min()
    maxIndex=df['flesch_index'].max()
    for i in range(length):
        index=df.loc[i,"flesch_index"]
        normalizedIndex=(index-minIndex)/(maxIndex-minIndex)
        df.loc[i,"normalized_flesch_index"]=normalizedIndex
    return df

**Function to normalize dale chall index**

In [ ]:
def normalize_dale_chall_index(df):
    df['normalized_dale_chall_index']=0
    length=len(df)
    minIndex=df['dale_chall_index'].min()
    maxIndex=df['dale_chall_index'].max()
    for i in range(length):
        index=df.loc[i,"dale_chall_index"]
        normalizedIndex=(index-minIndex)/(maxIndex-minIndex)
        df.loc[i,"normalized_dale_chall_index"]=normalizedIndex
    return df

**Function to compute final semantic score**

In [ ]:
def compute_final_score(df):
    df['final_semantic_score']=0
    length=len(df)
    for i in range(length):
        sub=df.loc[i,"subjectivity"]
        pol=df.loc[i,"normalized_polarity"]
        ent=df.loc[i,"normalized_entropy"]
        fle=df.loc[i,"normalized_flesch_index"]
        dal=df.loc[i,"normalized_dale_chall_index"]
        finalScore=(sub+pol+ent-fle-dal)/(5)
        df.loc[i,"final_semantic_score"]=finalScore
    return df